***Import Library's***

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

***Load the data***

In [4]:
df_origen = pd.read_csv('C:/Users/RC/Desktop/DS4A_workspace/git/Team35/Grupo35-DS4A/Final_Project/1.Script/1.Ext_Web_Scraping/OLX/Data_result_web_scrapping/OLX_Data.csv')

ParserError: Error tokenizing data. C error: Expected 35 fields in line 591, saw 36


***Eliminar duplicados***

In [336]:
train = df_origen.drop_duplicates().reset_index(drop=True)
n_rows,n_columns =train.shape
print(n_rows)

16553


In [337]:
train[train['Código web']=='100-M2504634'].to_csv('prueba2.csv')

***Limpieza de NAN***

In [277]:
data_count_null = train.isnull().sum().reset_index(name='q_nan')

# Variable identificadas como no utiles
columns_to_drop = ['Ancho','Características del Parqueadero','Con muebles','Depósitos','Fondo','Número de Ascensores',
                   'Número de Closets','Número de líneas teléfonicas','Terraza/Balcón','Tipo de Cortinas',
                   'Tipo de calentador','Tipo de estufa','Tipo de parqueadero','Tipo de piso en alcobas',
                   'Tipo de piso en comedor','Tipo de piso en estudio','Tipo de piso en sala','Tipo instalación de gas',
                   'Valor de arriendo','areaPrivada','businessTypeId','cityname','hdCompanyUrl','hdIsOcasional',
                   'hdnBusinessType','nameProperty','nomBarrio','nombreEmpresa','propertyTypeId','valorArriendo',
                   'Área Terraza/Balcón','Área privada','Habitaciones',
                  'Área construida','Área del lote','Baños']

#identifica los campos que tengan el 90% de sus 
for columns in data_count_null[data_count_null['q_nan']==(n_rows)]['index']:
    columns_to_drop.append(columns)

train = train.drop(columns_to_drop,axis='columns')

***Manejo de extras o complementos***

- Extra1: El campo viene con las descripciones de los compementos -- No se debe hacer cambios
- Extra2: Contiene si o no, se debe tomar el nombre del campo como el atributo de complemento
- Extra3: El campo contiene la descripción y dentro de la misma indica que tipo de extra es
- Extra4: El campo contiene la descripción pero es necesario complementar la información con el nombre del campo

In [304]:
train_test = train.copy()

In [305]:
extra2_descripcion = ['Ascensor','Cerca Centros Comerciales','Cerca Colegios / Universidades','Cerca Parques',
                      'Cerca Supermercados','Cerca Transporte Público','Conjunto cerrado','Cuarto de servicio',
                      'En interior y/o bloque','Esquinero','Estudio o biblioteca','Parqueadero cubierto',
                      'Zona de BBQ','Zona de lavanderia']

train_test['extra2'] = ''
for extra2 in extra2_descripcion:
    train_test[extra2] = train_test[extra2].apply(lambda x: extra2+',' if x == 'Si' else '')
    train_test['extra2'] = train_test['extra2']+train_test[extra2]
    #train_test = train_test.drop([extra2],axis='columns')


In [306]:
extra3_descripcion = ['Tipo comedor','Tipo de Casa','Tipo de acabado piso']

train_test['extra3'] = ''
for extra3 in extra3_descripcion:
    train_test[extra3] = train_test[extra3].apply(lambda x: x+',' if x==x else '')
    train_test['extra3'] = train_test['extra3']+train_test[extra3]
    train_test = train_test.drop([extra3],axis='columns')

In [307]:
extra4_descripcion = ['Tipo de Cocina','Vigilancia','Vista']

train_test['extra4'] = ''
for extra4 in extra4_descripcion:
    
    train_test[extra4] = train_test[extra4].apply(lambda x: extra4+' '+str(x)+',' if x==x else ' ')
    train_test['extra4'] = train_test['extra4']+train_test[extra4]
    train_test = train_test.drop([extra4],axis='columns')


In [315]:
train_test['extra'] = train_test['complemento']+train_test['extra2']+train_test['extra3']+train_test['extra4']


In [317]:
train = train_test.drop(['complemento','extra2','extra3','extra4'],axis='columns').copy()

***Homologacion de campos***

In [334]:
train[train['Código web']=='100-M2504634'].to_csv('prueba2.csv')

In [333]:
x = ['Parqueadero','numGaraje','Código web']
train[x].to_csv('prueba.csv')

for y in x:
    print("****",y)
    print(train[y].unique())

**** Parqueadero
['1.0' nan '2.0' '3.0' '4 o más' '2' '1' '3' '4+']
**** numGaraje
['1.0' '0.0' '2.0' '3.0' '4.0' nan '4+' '3' '2' '4' '0' '1']
**** Código web
['2571-M2012354' '920-42149736' '607-M2267033' ... '603-M2373742'
 '639-M4769' '9-2019050003']


In [ ]:
homologacion_columns = {'Category1':'tipo_inueble',
                        'Location3':'zona',
                        'Nombre común del barrio':'barrio1', 
                        'Nombre del barrio catastral':'sector_catastral', 
                        'description':'descripcion', 
                        'Price':'precio',
                        'ContractType':'tipo_contrato',
                        'areaConstruida':'area', 
                        'Address':'direccion', 
                        'Rooms':'numero_habitaciones', 
                        'numBanos':'num_banos',
                        'Tiempo de construido':'edad',
                        'Condition':'condicion', 
                        'Floor': 'piso_ubicacion', 
                        'AdministrationPrice':'precio_administracion',
                        'Estrato':'estrato',
                        'Garages':'num_garages',
                        'Extras':'extras', 
                        'latitude':'latitude',
                        'longitude':'longitude',
                        'InteriorFloors':'piso_interior',
                        'Código web':'id_inmueble'}

train = train.rename(columns = homologacion_columns)